In [ ]:
"""
Basic Blackjack Rules:

The goal of blackjack is to beat the dealer's hand without going over 21.
Face cards are worth 10. Aces are worth 1 or 11, whichever makes a better hand.
Each player starts with two cards, one of the dealer's cards is hidden until the end.
To 'Hit' is to ask for another card. To 'Stand' is to hold your total and end your turn.
If you go over 21 you bust, and the dealer wins regardless of the dealer's hand.
If you are dealt 21 from the start (Ace & 10), you got a blackjack.
Blackjack usually means you win 1.5 the amount of your bet. Depends on the casino.
Dealer will hit until his/her cards total 17 or higher.
Doubling is like a hit, only the bet is doubled and you only get one more card.
Split can be done when you have two of the same card - the pair is split into two hands.
Splitting also doubles the bet, because each new hand is worth the original bet.
You can only double/split on the first move, or first move of a hand created by a split.
You cannot play on two aces after they are split.
You can double on a hand resulting from a split, tripling or quadrupling you bet.

"""

In [155]:
from random import shuffle #shuffle cards
import time #dramatic effect for House card draws

#Generate a Deck with 52 cards without suits
def createDeck():
    Deck=[]
    
    faceValues=["A","K","Q","J"]
    for i in range(4): #4 suits
        for card in range(2,11): #number cards
            Deck.append(str(card))
        for card in faceValues: #face values
            Deck.append(card)
            
    shuffle(Deck)
    return Deck

#Define a Player class
class Player:
    def __init__(self,hand=[],money=100):
        self.hand=hand
        self.score=self.setScore()
        self.money=money
        self.bet=0
    
    def __str__(self):#when printing, display hand and score
        currentHand= "" #self.hand=["A","K"]
                        #print "A K "
        for card in self.hand:
            currentHand+=str(card)+" "
        
        finalStatus=currentHand+"score: "+str(self.score)
        return finalStatus
    
    def setScore(self): #Evaluate current hand and set score
        self.score=0
        facecardDict={"A":11,"K":10,"Q":10,"J":10,"10":10,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1}
        aceCounter=0
        
        for card in self.hand:
            self.score+=facecardDict[card]
            if card=="A":
                aceCounter+=1
            if self.score>21 and aceCounter!=0: #Aces can be 11 or 1
                self.score-=10
                aceCounter-=1
                
        return self.score
    
    def hit(self,card): #Add a card
        self.hand.append(card)
        self.score=self.setScore()
        
    def play(self,newHand): #play a hand
        self.hand= newHand
        self.score=self.setScore()
        
    def betMoney(self,amount): #Bet money
        self.money-= amount
        self.bet += amount
        
    def win(self,result): #win bet return
        if result == True:
            if self.score==21 and len(self.hand)==2:
                self.money += 2.5*self.bet
            else:
                self.money += 2*self.bet
            self.bet=0
        else:
            self.bet=0
            
    def hasBlackjack(self):
        if self.score==21 and len(self.hand)==2:
            return True
        else:
            return False
        
    def draw(self):
        self.money+=self.bet
        self.bet=0

def printHouse(House): #Initially first card of house hidden
    for card in range(len(House.hand)):
        if card==0:
            print("X",end=" ")
        elif card==len(House.hand)-1:
            print(House.hand[card])
        else:
            print(House.hand[card], end=" ")
        
cardDeck=createDeck()
Player1=Player(firstHand,money=100) #Can change amount of money here by adding "money=X"
House=Player(secondHand)
cardDeck=createDeck()

while Player1.money>0: #Can keep playing as long as player has money, default starting cash is 100
    if len(cardDeck)<20: #Shuffle deck if less than 20 card left
        cardDeck=createDeck()
    firstHand=[cardDeck.pop(),cardDeck.pop()] #giving player the first two top (bottom) cards
    secondHand=[cardDeck.pop(),cardDeck.pop()] 
    Player1.play(firstHand)
    House.play(secondHand)
    
    print("Money: "+str(Player1.money))

    while True: #Player can choose to keep betting, or just cash out to get the current money
        user_input1=input("Bet or Cash out (B/C) ")
        if user_input1.upper() == "C":
            print("You get "+str(Player1.money))
            break
        elif user_input1.upper() == "B": #If betting, input has to be numeric
            user_input2=input("Please enter your bet: ")
            if user_input2.isnumeric():
                Bet=round(float(user_input2),2) #currency 2 digit
                while Bet>Player1.money: #and have enough money to bet the amount
                    print("Invalid Entry")
                    print("Money: "+str(Player1.money))
                    try: #user might mistakenly enter a non-numeric bet
                        Bet=round(float(input("Please enter your bet: ")),2)
                    except: #wordaround
                        print("Bet should be numeric!")
                        pass
                break #break if bet is numeric and less than money
            else:
                print("Invalid Entry") #first bet is non-numeric
        else:
            print("Invalid Entry") #not "B" or "C"
        
    if user_input1.upper() == "C": #Player choose to cash out, break out loop, otherwise continue
        break
    
    Player1.betMoney(Bet) #Make a bet
    
    #1st Win Check
    if Player1.hasBlackjack():
        if House .hasBlackjack():
            print(Player1)
            Player1.draw()
            print("Both Blackjack!")#Draw with both Blackjack
        else:
            print(Player1)
            print("Blackjack!")
            Player1.win(True) #Player hit Blackjack before house
    else:#No win yet
        
        #Player's turn
        while(Player1.score<21): #No blackjack, options to hit card if no bust
            printHouse(House)
            action=input("Do you want another card (y/n): ")
            if action == "y":
                Player1.hit(cardDeck.pop()) #Gives the top (bottom) card to the player
                print(Player1)
            else: #any other input is a "n"
                break

        #House's turn
        while(House.score<16): #Once player's hand is settled, house starts to draw card until 16 points
            time.sleep(1) #Drama 1
            print(House)
            House.hit(cardDeck.pop())
            time.sleep(1) #Drama 2
        print(House)
        
        #2nd Win Check
        #Each sceneario explained by print statement
        if Player1.score>21:
            if House.score >21:
                Player1.draw()
                print("Both busted, draw")
            else: #Player bust
                Player1.win(False)
                print("You busted")
        elif Player1.score > House.score:
            Player1.win(True)
            print("You scored higher!")
        elif Player1.score==House.score:
            Player1.draw()
            print("Draw")
        else:
            if House.score>21:
                Player1.win(True)
                print("House Busted")
            else:
                Player1.win(False)
                print("Unfortunately you scored less")

#Player reach here only if cash out or bankrupt
if user_input1.upper() == "C":
    pass
else:
    print("Bankrupt!")

Money: 100
Bet or Cash out (B/C) b
Please enter your bet: 50
9 Q score: 19
X 5
Do you want another card (y/n): n
3 5 score: 8
3 5 A score: 19
Draw
Money: 100.0
Bet or Cash out (B/C) b
Please enter your bet: 50
3 9 score: 12
X 4
Do you want another card (y/n): n
6 4 score: 10
6 4 J score: 20
Unfortunately you scored less
Money: 50.0
Bet or Cash out (B/C) c
You get 50.0
